# Python + Pandas is love

L'idéal pour une rapport de données, la validation d'une intuition, vite fait, sur un coin de table, en mode oneshot / quick&dirty ? ~~Excel~~ [Pandas](https://pandas.pydata.org/) !

Nous allons suivre la génération d'un rapport à partir d'un _petit_ dump de données, ce qui va nous permettre de faire une visite guidée des fonctionnalités de pandas. Ce ne sera pas exhaustif, mais devrait vous permettre d'y penser la prochaine fois. 


## La base


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

### Serie

Une série, est une séquence de valeurs, de type homogène (toutes du même type : String, entiers, flotants, dates ... ), __ avec un index __

In [2]:
s = pd.Series([5,4,3,2,1,2,3,4,5])
s_datetimes = pd.date_range(start="2017-01-01", periods=86400, freq="2H").to_series()

#display(s.head())
s_datetimes.head()

2017-01-01 00:00:00   2017-01-01 00:00:00
2017-01-01 02:00:00   2017-01-01 02:00:00
2017-01-01 04:00:00   2017-01-01 04:00:00
2017-01-01 06:00:00   2017-01-01 06:00:00
2017-01-01 08:00:00   2017-01-01 08:00:00
Freq: 2H, dtype: datetime64[ns]

## Dataframe

Un Dataframe, c'est un tableau : 
- un ensemble de Series, nommées ( les colonnes)
- qui partagent un index


In [3]:
data = pd.DataFrame({"time": s_datetimes[:120].values,  # get rid of the index
                     "value": np.random.random(120),
                     "category": pd.Categorical(list("abcdef" * 20))})

print(data.info(()))
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
category    120 non-null category
time        120 non-null datetime64[ns]
value       120 non-null float64
dtypes: category(1), datetime64[ns](1), float64(1)
memory usage: 2.1 KB
None


,category,time,value
0,a,2017-01-01 00:00:00,0.845008
1,b,2017-01-01 02:00:00,0.189874
2,c,2017-01-01 04:00:00,0.169918
3,d,2017-01-01 06:00:00,0.579058
4,e,2017-01-01 08:00:00,0.469799


### Import/Export

L'idée étant évidemment d'importer nos données ici, et pas de les générer. Pandas supporte à peu près tout les fichiers formats de données courants :
- csv 
- excel 
- hdf5 
- json/msgpack/parquet

_mais aussi_  on peut créer un dataframe directement depuis une requête SQL, ou même Big Query.

Nous allons ici utiliser la lingua franca de la data ( rappel du contexte : on est sale, on est rapide, on est pas sexys ...) : le csv. 

In [4]:
clients = pd.read_csv("client_files.csv", delimiter=";", index_col=0, parse_dates=[1])
#display(clients.head(4))
events = pd.read_csv("events.csv", delimiter=";", index_col=0, parse_dates=True)
#display(events.info())
print("events is row x cols", events.shape)
print("columns are : ", events.columns)
print("time range:", events.index.min(), events.index.max())
display(events.head(4))

events is row x cols (21938, 4)
columns are :  Index(['client_uuid', 'type', 'doc_type', 'status'], dtype='object')
time range: 2017-01-01 04:07:02 2017-01-21 20:39:23


,client_uuid,type,doc_type,status
timestamp,,,,
2017-01-01 04:07:02,5ada8b5e-bb71-4bf7-b045-bb69956fa744,DOC,TAX_NOTICE,True
2017-01-01 04:07:05,5ada8b5e-bb71-4bf7-b045-bb69956fa744,DOC,PAYSLIP,True
2017-01-01 04:07:06,5ada8b5e-bb71-4bf7-b045-bb69956fa744,SIGNATURE,NaN,True
2017-01-01 05:32:01,c4837f20-8635-4736-baa9-86f1ca5991a9,DOC,CNI,False


## Let's go

### Les données :

Les clients remplissent des dossiers pour soucrire à des produits, ils doivent envoyer certains justificatifs qui sont
  validés automatiquement (ou pas). events enregistre les différentes soumissions de documents. (client_uuid, correspond à un dossier). comme la validation automatique peut échouer, le client resoumet ces pièces.

### Le probème
Nous voudrions compiler un peu des stats sur le nombre de soumission, par type de dossier ... juste à partir de la table des evenements.


### Requêtes simples

In [5]:
import datetime

cni_events = events[events.doc_type == "CNI"] # keep only CNI type docs
# keep only events occuring on january 10th. 
day_events = events[events.index.date == datetime.date(2017,1,10)]

print("%d events recorded on January 10th" % len(day_events))

# By Index . 
events.ix[14:16]

1006 events recorded on January 10th


/home/flo/code/python-pandas-talk/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


,client_uuid,type,doc_type,status
timestamp,,,,
2017-01-01 07:58:19,bec58651-0c28-4c15-a51e-162b69d4a3c0,DOC,RIB,True
2017-01-01 07:59:17,bec58651-0c28-4c15-a51e-162b69d4a3c0,SIGNATURE,NaN,True


In [6]:
# Pour des requêtes encore plus péchues 

events.query('doc_type=="RIB" & status & index > "20170110"').head()


,client_uuid,type,doc_type,status
timestamp,,,,
2017-01-10 00:52:53,cf730cf8-df07-421e-930d-96cf30b943d4,DOC,RIB,True
2017-01-10 02:35:46,aebc927f-c6e1-41b6-ab75-86e595b5d8e1,DOC,RIB,True
2017-01-10 07:19:55,807c85e3-ac99-4439-b331-e91263757bd0,DOC,RIB,True
2017-01-10 09:19:18,79e6e17c-c4bf-4e44-b830-f249a41af256,DOC,RIB,True
2017-01-10 10:18:33,e4f90caf-bfef-4083-b565-f01a4e85aead,DOC,RIB,True


### GroupBy

  Tableau croisé dynamique, c'est mieux si on en reste là.

In [7]:
# comptons les soumissions de documents par type/status. 
(events.groupby(('doc_type','status')) # groupage imbriqué 
       .count())   # fonction d'aggregation 


client_uuid  type
doc_type   status                   
CNI        False          1299  1299
           True           1881  1881
PAYSLIP    False          1172  1172
           True           1866  1866
RIB        False          1894  1894
           True           2854  2854
TAX_NOTICE False          2497  2497
           True           3719  3719

In [11]:
# utiliseons groupby pour "numéroter" les soumissions par utilisateurs

# cleanup
events['doc_type'] = events['doc_type'].fillna('NO_DOC')
events["timestamp"] = events.index # --- groupby transform pas très heureux avec les index

# groupons par dossier, puis chaque document
grouped_doc = events.groupby(('client_uuid','doc_type'))

# on transforme une colonne. ( Split - apply - combine)
events['doc_submission_count'] = grouped_doc["timestamp"].transform(np.argsort)
events['time_since_first'] = events.groupby('client_uuid')["timestamp"].transform(lambda ts: ts-ts[0])

events.head()

,client_uuid,type,doc_type,status,timestamp,doc_submission_count,time_since_first
timestamp,,,,,,,
2017-01-01 04:07:02,5ada8b5e-bb71-4bf7-b045-bb69956fa744,DOC,TAX_NOTICE,True,2017-01-01 04:07:02,0,00:00:00
2017-01-01 04:07:05,5ada8b5e-bb71-4bf7-b045-bb69956fa744,DOC,PAYSLIP,True,2017-01-01 04:07:05,0,00:00:03
2017-01-01 04:07:06,5ada8b5e-bb71-4bf7-b045-bb69956fa744,SIGNATURE,NO_DOC,True,2017-01-01 04:07:06,0,00:00:04
2017-01-01 05:32:01,c4837f20-8635-4736-baa9-86f1ca5991a9,DOC,CNI,False,2017-01-01 05:32:01,0,00:00:00
2017-01-01 05:33:45,c4837f20-8635-4736-baa9-86f1ca5991a9,DOC,CNI,True,2017-01-01 05:33:45,1,00:01:44
